In [9]:
pip list | grep -i langchain

langchain                         1.1.0
langchain-classic                 1.0.0
langchain-community               0.4.1
langchain-core                    1.1.0
langchain-groq                    1.1.0
langchain-openai                  1.1.0
langchain-text-splitters          1.0.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langgraph.prebuilt import create_react_agent

### Define Tools

In [2]:
@tool
def tavily_search(query: str) -> str:
    """Use this tool to search the web using Tavily. Best for general research."""
    try:
        response = tavily.search(query=query, max_results=5)
        return str(response)
    except Exception as e:
        return f"Tavily Search Error: {e}"

@tool
def publications_search(topic: str) -> str:
    """Search academic publications using CrossRef API (simple beginner version)."""
    try:
        url = f"https://api.crossref.org/works?query={topic}&rows=3"
        res = requests.get(url).json()
        items = res.get("message", {}).get("items", [])
        papers = []
        for item in items:
            title = item.get("title", ["No title"])[0]
            year = item.get("created", {}).get("date-parts", [[None]])[0][0]
            doi = item.get("DOI", "N/A")
            papers.append({"title": title, "year": year, "doi": doi})
        return str(papers)
    except Exception as e:
        return f"Publications API Error: {e}"

@tool
def evaluate_info(text: str) -> str:
    """Evaluate the usefulness of the gathered research."""
    return f"[Evaluation] Looks like useful information. Summary appears valid."

tools = [tavily_search, publications_search, evaluate_info]


## Initiate LLM and Agent

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = create_react_agent(llm, tools)

/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_56848/1109227244.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


## Start Research 

In [6]:
topic = "Attention is all you need?"

prompt = f"""
You are a Professional Researcher Agent.

Your workflow:
1. Create a clear research PLAN.
2. Use `tavily_search` to gather real web information.
3. Use `publications_search` to gather academic papers.
4. Use `evaluate_info` to assess the quality of the research.
5. Produce a final research report.

Topic to research: {topic}

Return final output in this structure:

### PLAN
### WEB RESEARCH (Tavily)
### PUBLICATION FINDINGS
### EVALUATION
### FINAL REPORT
"""


response = agent.invoke({"messages": f"Research the topic: {topic}"})

In [8]:
print(response["messages"][-1].content)

The phrase "Attention is all you need" is famously associated with a seminal paper in the field of machine learning and natural language processing, specifically regarding the Transformer model. Here are some relevant publications on the topic:

1. **Is Attention All You Need?** (2025) - [DOI: 10.1007/978-3-031-84300-6_13](https://doi.org/10.1007/978-3-031-84300-6_13)
   
2. **Attention Is All You Need** (2025) - [DOI: 10.65215/nxvz2v36](https://doi.org/10.65215/nxvz2v36)

3. **All You Need Is LSD** (2020) - [DOI: 10.5040/9781350101272.00000005](https://doi.org/10.5040/9781350101272.00000005)

The first two publications are directly related to the concept of attention mechanisms in neural networks, particularly in the context of the Transformer architecture, which has revolutionized the field of deep learning. The third publication appears to be unrelated to the topic of attention in machine learning.

If you would like more detailed information or specific aspects of the "Attention is